In [1]:
import os
import random
import time
import collections
import json

import numpy as np
import pandas as pd
import pandasql as pdsql
import miceforest as mf
from warnings import simplefilter
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils.util_preprocess import *
#from utils.softimpute import softimpute, cv_softimpute

# Load dataset

## 1. Initial dataset:   
----> df_arr, mapping

In [2]:
path_dataset = "/Users/kaiyuli/Documents/Research/DataMarket/_dataset/"
dataset = ['forest10']

# preprocess a dataset
# check the initialized schema manually first.
df = pd.read_csv(path_dataset + "TPCDS/original.csv")
#df['education_num'] = df['education_num'].astype('object')
df = df.drop('Unnamed: 0', axis=1)
# df_arr original table to vectors
#         integer remains int64
#         float remains float64
#         others to category
# mapping schema
df_arr, mapping = df_2_array(df)
# df_arr_new, mapping = df_2_array(df)

## 2.EXP settings

In [3]:
missing_rate = 0.3
pattern = 0
budget = 0.01

multiple_imputing_num = 10 # number of models for multiple imputing
discount_factor = 0.8

## 3. Incomplete Dataset
----> df_miss, x_miss, miss_rows, miss_cols

In [4]:
# do not deep copy df_miss or x_miss
# df_miss have the same schema with df_arr
#         but all numbers are float64
# df_miss and x_miss do not share the same part of memory
df_miss = mf.ampute_data(df_arr, perc=missing_rate, random_state=1991)
# df_miss = mf.ampute_data(df_arr, variables=[0, 1, 2, 9], perc=missing_rate, random_state=1991)
# x_miss = df_miss.to_numpy()
# miss_rows, miss_cols = np.where(np.isnan(x_miss))

In [5]:
df_miss.isna().sum().sum(), df_arr.isna().sum().sum()

(12415727, 1689764)

In [5]:
def generate_mar(df_input, missing_mapping):
    df_copy = df_input.copy()
    for pattern in missing_mapping:
        mask = df_copy[pattern['obs']] < pattern['delta']
        df_copy.loc[mask, pattern['miss']] = np.nan
    return df_copy

#df_miss = generate_mar(df, [{'obs':'Feature1', 'miss':'Feature2', 'delta':0.5}])

In [31]:
missing_mapping_mar_TPCDS = [{'obs':'ss_sales_price', 'miss':'ss_quantity', 'delta':15},
                  {'obs':'ss_sales_price', 'miss':'ss_wholesale_cost', 'delta':10},
                  {'obs':'ss_sales_price', 'miss':'ss_list_price', 'delta':10},
                  {'obs':'ss_ext_sales_price', 'miss':'ss_ext_discount_amt', 'delta':500},
                  {'obs':'ss_ext_sales_price', 'miss':'ss_ext_wholesale_cost', 'delta':400},
                  {'obs':'ss_ext_sales_price', 'miss':'ss_ext_tax', 'delta': 600},
                  {'obs':'ss_ext_sales_price', 'miss':'ss_ext_list_price', 'delta':800},
                  {'obs':'ss_net_paid', 'miss':'ss_coupon_amt', 'delta':200},
                  {'obs':'ss_net_paid', 'miss':'ss_net_paid_inc_tax', 'delta':1000},
                  {'obs':'ss_net_paid', 'miss':'ss_net_profit', 'delta':1000},]

In [55]:
missing_mapping_mnar_TPCDS = [
                  {'obs':'ss_wholesale_cost', 'miss':'ss_wholesale_cost', 'delta':20},
                  {'obs':'ss_list_price', 'miss':'ss_list_price', 'delta':10},
                  {'obs':'ss_sales_price', 'miss':'ss_sales_price', 'delta':20},
                  {'obs':'ss_ext_discount_amt', 'miss':'ss_ext_discount_amt', 'delta':500},
                  {'obs':'ss_ext_sales_price', 'miss':'ss_ext_sales_price', 'delta':300},
                  {'obs':'ss_ext_wholesale_cost', 'miss':'ss_ext_wholesale_cost', 'delta':700},
                  {'obs':'ss_net_paid', 'miss':'ss_coupon_amt', 'delta':200},
                  {'obs':'ss_net_paid', 'miss':'ss_net_paid_inc_tax', 'delta':1000},
                  {'obs':'ss_net_paid', 'miss':'ss_net_profit', 'delta':1200}
                  ]

In [56]:
df_miss = generate_mar(df_arr, missing_mapping_mnar_TPCDS)

In [7]:
df.isna().sum().sum(), df_arr.isna().sum().sum(), df_miss.isna().sum().sum()/df_miss.size

(1689764, 1689764, 0.3315701280365265)

In [6]:
df_miss.dtypes

ss_quantity              float64
ss_wholesale_cost        float64
ss_list_price            float64
ss_sales_price           float64
ss_ext_discount_amt      float64
ss_ext_sales_price       float64
ss_ext_wholesale_cost    float64
ss_ext_list_price        float64
ss_ext_tax               float64
ss_coupon_amt            float64
ss_net_paid              float64
ss_net_paid_inc_tax      float64
ss_net_profit            float64
dtype: object

--> queries

In [8]:
queries = []
with open(path_dataset + "TPCDS/queries.json") as f:
    for eachline in f:
        query = json.loads(eachline.strip('\n'))
        queries.append(query['data'])
W = generate_weights(len(queries))

In [9]:
len(queries)

18

# Real Implementation

In [10]:
def query_on_df(qs, df):
    df_query = df
    for q in qs:
        if q['width']==0:
            df_query = df_query[df_query[q['col']]==q['center']]
        else:
            df_query = df_query[(df_query[q['col']]>=q['center']-q['width']/2)&(df_query[q['col']]<=q['center']+q['width']/2)]

    return df_query


# input: multiple imputors, a single query
# output: line numbers of the top-k cells to acquire with maximum utility
# input: multiple imputors, multiple queries
# output: line numbers of the top-k cells to acquire with maximum utility
def get_uncertainty_score(kernel, queries, k=100):
    acquire_task = []
    for query in queries:
        acquire_task_freq = []
        for i in range(kernel.dataset_count()):
            acquire_task_freq.extend(query_on_df(query, kernel.complete_data(i)).index.values.tolist())
        acquire_task_freq.sort()
        counter = collections.Counter(acquire_task_freq)
        
        uncertainty_N = kernel.dataset_count()
        acq_task = [(item, count*(uncertainty_N-count)) for item, count in counter.items() if count!=0]
        
        acquire_task.extend(acq_task)
    
    freq_dict = {}
    for t in acquire_task:
        if t[0] not in freq_dict:
            freq_dict[t[0]] = 0
        freq_dict[t[0]] += t[1]
    
    #acq_task = counter.most_common(k)
    return list(freq_dict.items())

def uncertainty_acquisition(df_miss, df_arr, budget, queries, W):
    df_miss_copy = df_miss.copy()
    budget_units = budget*df_miss.isna().sum().sum() # it is not a integer
    
    # Create kernels. 
    kernel = mf.ImputationKernel(
      data=df_miss_copy,
      datasets=5,
      save_all_iterations=False,
      random_state=1991,
    )

    # Run the MICE algorithm for 3 iterations on each of the datasets
    kernel.mice(3, verbose=True, min_data_in_leaf = 6)

    
    acq_task = get_uncertainty_score(kernel, queries, int(budget_units/0.8))
    row_indexes = [t[0] for t in acq_task]
    df_needed_imputed = df_miss_copy.loc[row_indexes]
    costs = get_costs(df_needed_imputed)
    
    util_per_cost = [(acq_task[i][0], costs[i]/acq_task[i][1]) for i in range(len(costs)) if acq_task[i][1]!=0]
    sorted_util_per_cost = sorted(util_per_cost, key=lambda x: x[1])
    
    additional_indexes = random.sample(list(set(list(df_arr.index))-set(row_indexes)), int(budget_units))
    additional_util_per_cost = [(_, 120000) for _ in additional_indexes]
    sorted_util_per_cost.extend(additional_util_per_cost)

    i = 0       
    while budget_units>0:
        if i>=len(sorted_util_per_cost):
            break
        row_ind = sorted_util_per_cost[i][0]
        i+=1
        miss_cols = np.where(df_miss_copy.iloc[row_ind].isna())[0]
        for col in miss_cols:
            if col in [0, 1, 2, 9]:
                df_miss_copy.iat[row_ind,col] = df_arr.iat[row_ind,col]
                budget_units -= 1
    
    return df_miss_copy




# input: multiple imputors, a single query
# output: line numbers of the top-k cells to acquire with maximum utility
# input: multiple imputors, multiple queries
# output: line numbers of the top-k cells to acquire with maximum utility
def get_utility_score(kernel, queries, k=100):
    acquire_task = []
    for query in queries:
        for i in range(kernel.dataset_count()):
            acquire_task.extend(query_on_df(query, kernel.complete_data(i)).index.values.tolist())
    acquire_task.sort()
    counter = collections.Counter(acquire_task)
    # Get items with non-zero frequency
    acq_task = [(item, count) for item, count in counter.items() if count != 0]

    #acq_task = counter.most_common(k)
  
    return acq_task

def greedy_and_improve_acquisition(df_miss, df_arr, budget, queries, W):
    df_miss_copy = df_miss.copy()
    #_rows, _cols = np.where(np.isnan(df_miss.to_numpy()))
    budget_units = budget*df_miss.isna().sum().sum() # it is not a integer
    
    # Create kernels. 
    kernel = mf.ImputationKernel(
      data=df_miss_copy,
      datasets=5,
      save_all_iterations=False,
      random_state=1991,
    )

    # Run the MICE algorithm for 3 iterations on each of the datasets
    kernel.mice(3, verbose=True, min_data_in_leaf = 6)
    
    acq_task = get_utility_score(kernel, queries, int(budget_units/0.8))
    row_indexes = [t[0] for t in acq_task]
    df_needed_imputed = df_miss_copy.loc[row_indexes]
    costs = get_costs(df_needed_imputed)
    
    util_per_cost = [(acq_task[i][0], costs[i]/acq_task[i][1]) for i in range(len(costs)) if acq_task[i][1]!=0]
    sorted_util_per_cost = sorted(util_per_cost, key=lambda x: x[1])
    
    additional_indexes = random.sample(list(set(list(df_arr.index))-set(row_indexes)), int(budget_units))
    additional_util_per_cost = [(_, 120000) for _ in additional_indexes]
    sorted_util_per_cost.extend(additional_util_per_cost)

    i = 0
    while budget_units>0:
        if i>=len(sorted_util_per_cost):
            break
        row_ind = sorted_util_per_cost[i][0]
        i+=1
        miss_cols = np.where(df_miss_copy.iloc[row_ind].isna())[0]
        for col in miss_cols:
            if col in [0, 1, 2, 9]:
                df_miss_copy.iat[row_ind,col] = df_arr.iat[row_ind,col]
                budget_units -= 1
        
#             df_miss_copy.iat[row_ind,col] = df_arr.iat[row_ind,col]
#         budget_units -= len(miss_cols)*(0.99**len(miss_cols))
    
    return df_miss_copy

# def greedy_and_improve_acquisition(df_miss, df_arr, budget, queries, W):
#     df_miss_copy = df_miss.copy()
#     _rows, _cols = np.where(np.isnan(df_miss.to_numpy()))
#     budget_units = budget*df_miss.isna().sum().sum() # it is not a integer
    
#     # Create kernels. 
#     kernel = mf.ImputationKernel(
#       data=df_miss_copy,
#       datasets=5,
#       save_all_iterations=False,
#       random_state=1991,
#     )

#     # Run the MICE algorithm for 3 iterations on each of the datasets
#     kernel.mice(3, verbose=True, min_data_in_leaf = 6)
    
#     acq_task = get_utility_score(kernel, queries, int(budget_units/0.8))
#     row_indexes = [t[0] for t in acq_task]
#     df_needed_imputed = df_miss_copy.loc[row_indexes]
#     costs = get_costs(df_needed_imputed)
    
#     util_per_cost = [(acq_task[i][0], costs[i]/acq_task[i][1]) for i in range(len(costs))]
#     sorted_util_per_cost = sorted(util_per_cost, key=lambda x: x[1])
    
#     additional_indexes = random.sample(list(set(list(df_arr.index))-set(row_indexes)), int(budget_units))
#     additional_util_per_cost = [(_, 10000) for _ in additional_indexes]
#     sorted_util_per_cost.extend(additional_util_per_cost)

#     i = 0
#     while budget_units>0:
#         row_ind = sorted_util_per_cost[i][0]
#         i+=1
#         miss_cols = np.where(df_miss_copy.iloc[row_ind].isna())[0]
#         for col in miss_cols:
#             df_miss_copy.iat[row_ind,col] = df_arr.iat[row_ind,col]
#         budget_units -= len(miss_cols)*(0.99**len(miss_cols))
    
#     return df_miss_copy

def query_by_rounds(df_miss, df_arr, budgets, queries, W):
    df_miss_copy = df_miss.copy()
    for budget in budgets:
        df_miss_copy = greedy_and_improve_acquisition(df_miss_copy, df_arr, budget, queries, W)
    
    return df_miss_copy

def get_acc(queries, acquired_df_miss, df_arr, W):
    
    formalize_df(acquired_df_miss, mapping, df.keys().to_list())
    
    pres = 0
    rs = 0 
    # no difference in evaluation parts
    for i, query in tqdm(enumerate(queries)):
        # get the dataframe of acquired answer and ground truth
        data_product = query_on_df(query, acquired_df_miss)
        ground_truth = query_on_df(query, df_arr)
        
        # get precision
        if len(ground_truth)==0:
            precision = 1
            rmse = 1
        else:
            ground_truth_indexes = set(ground_truth.index.values.tolist())
            data_product_indexes = set(data_product.index.values.tolist())
            intersect = ground_truth_indexes.intersection(data_product_indexes)
            precision = len(intersect)/len(ground_truth_indexes)

            # get rmse = precision * RMSE
            indexes_lst = list(intersect)
            imputing_error = get_avg_error(df_arr.iloc[indexes_lst], acquired_df_miss.iloc[indexes_lst], mapping)
            rmse = precision*(1-imputing_error)
        
        pres += precision*W[i]
        rs += rmse*W[i]
        
    return pres, rs

In [12]:
times = []
precisions = []
rmses = []

times_imp = []
precisions_imp = []
rmses_imp = []

idx = []
for budget in [0]:
    
    # step 1. modify budget
    budget = (budget+1)*0.01 
    idx.append(budget)
    #budget /= discount_factor
    
    # step 2. change methods
    # acquisition results : acquired_df_miss
    st = time.time()
    acquired_df_miss = query_by_rounds(df_miss, df_arr, [budget/3, budget/3, budget/3], queries, W)
    et = time.time()
    
    acquired_df_miss_improved = query_by_rounds(acquired_df_miss, df_arr, [0.3 * budget], queries, W)
    et1 = time.time()
    
    pres, rs = get_acc(queries, acquired_df_miss, df_arr, W)
    pres_imporved, rs_improved = get_acc(queries, acquired_df_miss_improved, df_arr, W)
    
    times.append(et-st)
    precisions.append(pres)
    rmses.append(rs)
    
    times_imp.append(et1-st)
    precisions_imp.append(pres_imporved)
    rmses_imp.append(rs_improved)
    print(budget)


result = pd.DataFrame({"time": times, "precision": precisions, "rmse":rmses}, index=idx)
result.to_csv("Greedy.csv")  # step 3. change the file name

result = pd.DataFrame({"time": times_imp, "precision": precisions_imp, "rmse":rmses_imp}, index=idx)
result.to_csv("Improved.csv") 

In [11]:
precisions = []
rmses = []
for budget in [0]:
    budget = (budget+1)*0.01 
    num_rounds = 20  
    df_miss_copy = df_miss.copy()
    for i in range(num_rounds):
        # df_miss_copy = uncertainty_acquisition(df_miss_copy, df_arr, budget/num_rounds, queries, W)
        df_miss_copy = greedy_and_improve_acquisition(df_miss_copy, df_arr, budget/num_rounds, queries, W)
        
        df_miss_copy_copy = df_miss_copy.copy()
        pres, rs = get_acc(queries, df_miss_copy_copy, df_arr, W)
        print(pres)
        precisions.append(pres)
        rmses.append(rs)

Initialized logger with name mice 1-3
Dataset 0
1  | ss_net_paid | ss_sales_price | ss_coupon_amt | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_list_price | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_wholesale_cost | ss_ext_sales_price | ss_quantity
2  | ss_net_paid | ss_sales_price | ss_coupon_amt | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_list_price | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_wholesale_cost | ss_ext_sales_price | ss_quantity
3  | ss_net_paid | ss_sales_price | ss_coupon_amt | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_list_price | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_wholesale_cost | ss_ext_sales_price | ss_quantity
Dataset 1
1  | ss_net_paid | ss_sales_price | ss_coupon_amt | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_list_price | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_wholesale_cost | ss_ext_sales_price | ss

18it [00:00, 38.51it/s]


0.5410335395904831
Initialized logger with name mice 1-3
Dataset 0
1  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_quantity | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
2  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_quantity | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
3  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_quantity | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
Dataset 1
1  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_quantity | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_

18it [00:00, 35.45it/s]


0.5605514167580176
Initialized logger with name mice 1-3
Dataset 0
1  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_quantity | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
2  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_quantity | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
3  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_quantity | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
Dataset 1
1  | ss_coupon_amt | ss_list_price | ss_wholesale_cost | ss_quantity | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_

...

0.828696460384769
Initialized logger with name mice 1-3
Dataset 0
1  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
2  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
3  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
Dataset 1
1  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_i

18it [00:00, 38.26it/s]


0.8346438567974594
Initialized logger with name mice 1-3
Dataset 0
1  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
2  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
3  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_inc_tax | ss_ext_sales_price
Dataset 1
1  | ss_quantity | ss_wholesale_cost | ss_list_price | ss_coupon_amt | ss_net_paid | ss_sales_price | ss_ext_discount_amt | ss_ext_list_price | ss_ext_wholesale_cost | ss_net_profit | ss_ext_tax | ss_net_paid_

18it [00:00, 39.82it/s]

0.840297587931968


In [15]:
hybrid = precisions

In [23]:
uncertainty = precisions

In [22]:
hybrid

[0.557925679218539,
 0.571628850358341,
 0.5979921964086085,
 0.6076783490826523,
 0.628133119396837,
 0.6529898159746595,
 0.6656047462971022,
 0.6923443637520283,
 0.7010100996708942,
 0.7214353549940791,
 0.7297389645275586,
 0.7455587483760026,
 0.7545805420773513,
 0.7697610195631631,
 0.7784372935745261,
 0.79320713771285,
 0.8003496112157444,
 0.8101099250548035,
 0.8161727298085587,
 0.8249552550780637]

In [24]:
df_output = pd.DataFrame({'Hybrid':hybrid, 'uncertainty':uncertainty})

In [25]:
df_output.to_csv("TPCDS-utility.csv")

In [12]:
df_output = pd.read_csv("TPCDS-utility.csv")

In [16]:
df_output = df_output.drop("Unnamed: 0", axis=1)

In [17]:
df_output['utility'] = precisions

In [19]:
df_output.to_csv("TPCDS-utility.csv", index=None)

In [20]:
len(queries)

18

In [24]:
cnt = 0
for query in queries:
    df_tmp = query_on_df(query, df_arr)
    cnt += len(df_tmp)

In [25]:
cnt

44646

In [26]:
1-18/44646

0.9995968283832818

In [27]:
0.9*0.9*0.96*0.3

0.23328000000000002

In [28]:
0.9*0.8*0.04*0.3

0.00864

In [29]:
len(df)

2880404

In [33]:
s= 1
for _ in list(df_arr.nunique()):
    s *= _

,ss_quantity,ss_wholesale_cost,ss_list_price,ss_sales_price,ss_ext_discount_amt,ss_ext_sales_price,ss_ext_wholesale_cost,ss_ext_list_price,ss_ext_tax,ss_coupon_amt,ss_net_paid,ss_net_paid_inc_tax,ss_net_profit
0,79.0,11.41,18.71,2.80,99.54,221.20,901.39,1478.09,6.08,99.54,121.66,127.74,-779.73
1,37.0,63.63,101.17,41.47,46.03,1534.39,2354.31,3743.29,59.53,46.03,1488.36,1547.89,-865.95
2,99.0,80.52,137.68,83.98,0.00,8314.02,7971.48,13630.32,0.00,0.00,8314.02,8314.02,342.54
3,14.0,57.37,76.30,6.10,0.00,85.40,803.18,1068.20,0.00,0.00,85.40,85.40,-717.78
4,100.0,25.08,36.86,0.73,0.00,73.00,2508.00,3686.00,6.57,0.00,73.00,79.57,-2435.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880399,41.0,18.00,34.02,14.62,0.00,599.42,738.00,1394.82,11.98,0.00,599.42,611.40,-138.58
2880400,65.0,35.22,53.18,35.09,0.00,2280.85,2289.30,3456.70,182.46,0.00,2280.85,2463.31,-8.45
2880401,13.0,61.07,64.12,62.19,0.00,808.47,793.91,833.56,64.67,0.00,808.47,873.14,14.56
2880402,86.0,83.78,150.80,150.80,0.00,12968.80,7205.08,12968.80,0.00,0.00,12968.80,12968.80,5763.72


In [37]:
df_arr.nunique()

ss_quantity                 100
ss_wholesale_cost          9901
ss_list_price             19727
ss_sales_price            18686
ss_ext_discount_amt      209365
ss_ext_sales_price       411391
ss_ext_wholesale_cost    380588
ss_ext_list_price        578761
ss_ext_tax                78652
ss_coupon_amt            209365
ss_net_paid              463256
ss_net_paid_inc_tax      618874
ss_net_profit            566151
dtype: int64